# <center> Post Test prediction

# Importing libraries

Importing the various neccessary libraries. The ones being used here are:
* numpy
* pandas
* seaborn
* matplotlib
* scikit learn
* matplotlib
* time

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
import time

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

%matplotlib inline
warnings.filterwarnings('ignore')

# Importing data

Reading the _csv_ data into a _pandas dataframe_

In [ ]:
data = pd.read_csv('../input/predict-test-scores-of-students/test_scores.csv')

Viewing the data using the *head* function

In [ ]:
data.head()

Since each student's student id is unique, the same will be used as index

In [ ]:
data=data.set_index('student_id')

# EDA

Checking the datatype and size of each column

In [ ]:
data.info()

Grouping the data by school type, school setting, and tecahing method hen seeing the median for number of students, pretest marks aand postest marks

In [ ]:
#display(data.groupby('teaching_method').mean())
display(data.groupby(['school_type','school_setting','teaching_method']).median())

Distribution of marks by teaching method

In [ ]:
px.histogram(data,x='posttest',color="teaching_method",color_discrete_map={'Standard':'lightgray','Experimental':'darkgray'})

In [ ]:
nonstandard = pd.DataFrame(data[data['teaching_method']!='Standard'].groupby('school').count()['n_student']).rename({'n_student':'Non Standard'},axis=1)
standard = pd.DataFrame(data[data['teaching_method']=='Standard'].groupby('school').count()['n_student']).rename({'n_student':'Standard'},axis=1)

schools = pd.concat([standard,nonstandard],axis=1)
schools = schools.replace(np.nan, 0)
schools['total']=schools['Standard']+schools['Non Standard']
schools=schools.sort_values(by='total',ascending=False).reset_index()
#schools

In [ ]:
fig = px.bar(schools, x="school", y=["Standard",'Non Standard'],title="Number of Students in each school")
fig.show()

In [ ]:
fig = px.sunburst(data.groupby(['school_type','school_setting','teaching_method','gender']).count().reset_index(), 
                  path=['school_setting','school_type','teaching_method'],
                  color_discrete_map={'Suburban':'black', 'Urban':'gold', 'Rural':'darkblue'},
                  values='n_student',title="Distributtion by setting, type and teaching_method",
                  height=500)
fig.show()

In [ ]:
fig = px.scatter(data, x="pretest", y="posttest", trendline="ols",height=500,width=1000,title='Relation between posttest and pretest marks')
fig.show()

In [ ]:
plt.figure(figsize=(10,10))
fig = px.scatter(data, x="pretest", y="posttest" ,height=500,width=1000,facet_col='teaching_method',color="school_type",
                title='Differnece between posttest and pretest marks of children based on School type')
fig.show()

In [ ]:
plt.figure(figsize=(10,10))
fig = px.scatter(data, x="pretest", y="posttest" ,height=500,width=1000,facet_col='teaching_method',color="school_setting",
                title='Differnece between posttest and pretest marks of children based on School Setting')
fig.show()

In [ ]:
plt.figure(figsize=(10,10))
fig = px.scatter(data, x="pretest", y="posttest" ,height=500,width=1000,color='gender',facet_col="teaching_method",
                title='Differnece between posttest and pretest marks of children based on gender')
fig.show()

In [ ]:
plt.figure(figsize=(10,10))
fig = px.scatter(data, x="pretest", y="posttest" ,height=500,width=1000,facet_col='teaching_method',color="lunch",
                title='Differnece between posttest and pretest marks of children based on free lunch')
fig.show()

In [ ]:
fig = px.scatter(data, x="n_student", y="posttest" ,height=400,width=900,color='teaching_method',trendline='lowess',title='Relation between marks and number of student per class')
fig.show()

# One Hot Encoding

All the non numeric values are converted to numeric in order for easy deployment of ML models.

In [ ]:
from sklearn.preprocessing import LabelEncoder
to_be_encoded = ['school','school_setting','school_type','classroom','teaching_method','gender','lunch']
label_encoder = LabelEncoder()
dfs = []
for i in to_be_encoded:
    temp = pd.DataFrame({'Before Encoding':data[i].unique(),'After Encoding':label_encoder.fit_transform(data[i].unique())})
    dfs.append([temp.sort_values(by=['After Encoding']),i])
    data[i] = label_encoder.fit_transform(data[i])
for i in dfs:
    print(i[1])
    display(i[0])
    print('\n')

# Feature Splitting

Selecting the columns to be taken as feature set, _X_, and target value, _Y_

In [ ]:
X = data.drop('posttest',axis=1)
Y = data['posttest']

In [ ]:
Y

# Feature Scaling

Due the large range of pretest values and n_stufdents (continuous variale), they are scaled down using the standard scaler.<br>
Othe features are ot scaled down as they are catergorical variables

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X[['pretest','n_student']] = sc.fit_transform(X[['pretest','n_student']])

# Test Train split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=52)

# Model Selection

In [ ]:
# Lasso regresor
from sklearn.linear_model import Lasso
lasso=Lasso()

#ElasticNet
from sklearn.linear_model import ElasticNet
enet=ElasticNet()

#Decision Tree
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor()

#KNR
from sklearn.neighbors import KNeighborsRegressor
knr=KNeighborsRegressor()

#Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor
gbr=GradientBoostingRegressor()

#MLP
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(random_state=1, max_iter=500)

# Linear Regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

#Random Forest
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

#RandomForest with Grid Search
n_estimators = [int(x) for x in np.arange(10,500,2)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.arange(1,50,1)]
max_depth.append(None)
min_samples_split = [int(x) for x in np.arange(1,50,1)]
min_samples_leaf = [int(x) for x in np.arange(1,10,1)]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
models = [lr,rf,rf_random,mlp,lasso,enet,gbr,dtr,knr]

In [ ]:
from sklearn import metrics
model_acc = []
model_mae = []
model_mse = []
model_rmse = []
model_time = []

for i in models:
    start=time.time()
    i.fit(X_train,Y_train)
    stop=time.time()
    pred = i.predict(X_test)
    h=[]
    for j in pred:
        h.append(round(j,0))
    model_acc.append(round((i.score(X_test, Y_test)*100),2))
    model_mae.append(round((metrics.mean_absolute_error(Y_test, h)),2))                 
    model_mse.append(round((metrics.mean_squared_error(Y_test, h)),2))
    model_rmse.append(round(np.sqrt(metrics.mean_squared_error(Y_test, h)),2))
    model_time.append(stop-start)
models = pd.DataFrame({'Models':models,'Accuracy':model_acc,'MAE':model_mae,'MSE':model_mse,'RMSE':model_rmse,'Time (s)':model_time})

In [ ]:
models = models.sort_values(by=['Accuracy'],ascending=False).reset_index().drop('index',axis=1)
best = models['Models'][0]
models['Models']=models['Models'].astype(str).str.split("(", n = 2, expand = True)[0]
models

In [ ]:
print('Thus,the best model is',models['Models'][0])
print('Accuracy:',models['Accuracy'][0])
print('MAE:',models['MAE'][0])
print('MSE:',models['MSE'][0])
print('RMSE:',models['RMSE'][0])

pred = best.predict(X_test)
h=[]
for i in pred:
    h.append(round(i,0))
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.scatter(Y_test,h)
plt.subplot(1,2,2)
sns.distplot((Y_test-h),bins=150,axlabel = "error")